# Use the same code --amrit i think this is much better

In [1]:
pip install datasets

In [1]:
from pathlib import Path
import os
import json
import torch
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    TrOCRProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from torchvision import transforms

# Use my drive amrit --> save every model  in the same folder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
CROPPED_DIR = "/content/drive/MyDrive/Gadulla/Final_Cropp_Image/whited_amount_digits"
LABELS_DIR = "/content"
MODEL_BASE_DIR = "/content/Final_models_trained"
FIELD = "amount_digits"

In [4]:
label_path = os.path.join(LABELS_DIR, f"{FIELD}_labels.json")
with open(label_path, "r", encoding="utf-8") as f:
    annotations = json.load(f)

In [5]:
annotations = [
    item for item in annotations
    if os.path.exists(os.path.join(CROPPED_DIR, item["image"])) and item["text"].strip().lower() != "null"
]# do not remove this--> it will help us to remove samples where file does not exist or text is null

In [6]:
train_items, val_items = train_test_split(annotations, test_size=0.2, random_state=42)

# Building hugging face dataset

In [7]:
def build_dataset(items):
    dataset_dict = {"image": [], "text": []}
    for item in items:
        dataset_dict["image"].append(os.path.join(CROPPED_DIR, item["image"]))
        dataset_dict["text"].append(item["text"])
    return Dataset.from_dict(dataset_dict)

train_dataset = build_dataset(train_items)
val_dataset = build_dataset(val_items)


In [8]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten") #--> we can have its higher version? should we go for it?
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout

In [9]:
# Augmentation
augment = transforms.Compose([
    transforms.RandomRotation(2),
    transforms.ColorJitter(brightness=0.1, contrast=0.1)
])

# Imp: here i think this one is better

In [12]:
def preprocess(examples):
    images = [Image.open(image_path).convert("RGB") for image_path in examples["image"]]
    images = [augment(image) for image in images]
    pixel_values = processor(images, return_tensors="pt").pixel_values
    input_ids = processor.tokenizer(
        examples["text"],
        padding="max_length",
        max_length=16,
        truncation=True,
        return_tensors="pt"
    ).input_ids

    return {
        "pixel_values": pixel_values,
        "labels": input_ids
    }

In [13]:
train_dataset = train_dataset.map(preprocess, batched=True, batch_size=16, remove_columns=['image'])
val_dataset = val_dataset.map(preprocess, batched=True, batch_size=16, remove_columns=['image'])
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")

Map:   0%|          | 0/2870 [00:00<?, ? examples/s]

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

In [14]:
# Collate Function
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch],
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )
    decoder_input_ids = labels[:, :-1].contiguous()
    labels_shifted = labels[:, 1:].clone()
    labels_shifted[labels_shifted == processor.tokenizer.pad_token_id] = -100
    return {
        "pixel_values": pixel_values,
        "decoder_input_ids": decoder_input_ids,
        "labels": labels_shifted
    }


# This one is better Shivanshu

In [15]:
# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(MODEL_BASE_DIR, FIELD),
    per_device_train_batch_size=8,
    num_train_epochs=10,
    learning_rate=3e-5,  #  3e-5 is good for date amrit
    save_strategy="no",
    logging_dir=f"./logs/{FIELD}",
    predict_with_generate=True,
    fp16=True,
    report_to="none",
    label_smoothing_factor=0.1,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
    data_collator=collate_fn
)


<ipython-input-15-c0b413a6d266>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [16]:
trainer.train()

Step,Training Loss
500,2.574600
1000,1.889100
1500,1.702200
2000,1.573500
2500,1.500700
3000,1.457400
3500,1.443600


TrainOutput(global_step=3590, training_loss=1.7270825781862051, metrics={'train_runtime': 2170.5039, 'train_samples_per_second': 13.223, 'train_steps_per_second': 1.654, 'total_flos': 2.147578498158428e+19, 'train_loss': 1.7270825781862051, 'epoch': 10.0})

In [17]:
# Save this is your drive as well ass if u want to download it, depends on you
model.save_pretrained(os.path.join(MODEL_BASE_DIR, FIELD))
processor.save_pretrained(os.path.join(MODEL_BASE_DIR, FIELD))

[]

# Date format--> amrit

In [18]:
def format_date(pred: str) -> str:
    pred = pred.strip().replace(" ", "").replace("/", "")
    if len(pred) == 8 and pred.isdigit():
        return f"{pred[:2]}/{pred[2:4]}/{pred[4:]}"
    return pred

# Do not run this I will tell you when we have to do evaluation-->Shivanshu

In [19]:
def evaluate_on_dataset(dataset, split_name):
    correct = 0
    total = 0
    for example in tqdm(dataset, desc=f" Evaluating {FIELD} on {split_name}"):
        image_tensor = example['pixel_values'].unsqueeze(0).to(model.device)
        label_text = processor.tokenizer.decode(example['labels'], skip_special_tokens=True)
        label_text = label_text.strip().replace(" ", "")

        with torch.no_grad():
            outputs = model.generate(
                image_tensor,
                max_length=16,
                num_beams=5,
                early_stopping=True
            )
            prediction = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        formatted_pred = format_date(prediction)
        if formatted_pred == label_text:
            correct += 1
        total += 1

    accuracy = correct / total * 100
    print(f"{split_name} Accuracy for {FIELD}: {accuracy:.2f}%")

evaluate_on_dataset(train_dataset, "train")
evaluate_on_dataset(val_dataset, "validation")

 Evaluating amount_digits on train: 100%|██████████| 2870/2870 [07:12<00:00,  6.63it/s]


train Accuracy for amount_digits: 99.06%


 Evaluating amount_digits on validation: 100%|██████████| 718/718 [01:48<00:00,  6.59it/s]

validation Accuracy for amount_digits: 91.09%


In [20]:
MODEL_BASE_DIR = "/content/drive/MyDrive/FinalTrained_model_directory"

model.save_pretrained(os.path.join(MODEL_BASE_DIR, FIELD))
processor.save_pretrained(os.path.join(MODEL_BASE_DIR, FIELD))

[]

# Nothing to be changed ... This parametes are good for amount_digits and amount in words, they are good .. performing better compare to other models-- Amrit